In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

In [ ]:
# aak_ge takes a while. chokes during feature selection COAD even with 5
for target in config.prediction_targets[:]:
    data, files = load.loadAll(includeStage=(target=="stage"), sameSamples=True, skipGenes=False)
    for sampling in config.sampling[:]:
        for selection in config.selection_types[:2]:
            # pred.runExperiments(data[1:2], files[1:2], target=target, sampling=sampling, selection=selection)
            
            for parity in config.modality_parities[:]:
                enforce_modality_parity = (parity == "parity")

                # Only run for overlap set if you run with parity
                if enforce_modality_parity:
                    pred.runExperiments(data[-1:], files[-1:], target=target, sampling=sampling, selection=selection, modality_selection_parity=enforce_modality_parity)
                else:
                    pred.runExperiments(data[:], files[:], target=target, sampling=sampling, selection=selection, modality_selection_parity=enforce_modality_parity)

In [ ]:
data, files = load.loadAll(includeStage=True, sameSamples=True)

In [ ]:
pred.runExperiments(data[:], files[:], target="stage", sampling="random_sampling")

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, log_loss
import numpy as np

d = data[2]
d = load.attachStageStatus(d)
x, y = pr.splitData(d, target="stage", project="ESCA")
print(y)
model = LinearRegression()
# pred.runRandomSampling(x, y, model)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, stratify=y, random_state=42)

model.fit(x_train, y_train)
y_predicted = model.predict(x_test)
y_predicted_rounded = np.rint(y_predicted)
y_predicted_rounded
# y_predicted
y.dtype


In [ ]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(random_state=0)
model.fit(x_train, y_train)


In [ ]:
model.coef_
# features_with_coefficients = pd.DataFrame(zip(x_train.columns, model.coef_))
features_with_coefficients = pd.DataFrame({"feature":x_train.columns,"coefficients":np.transpose(model.coef_)})
features_with_coefficients_abs = features_with_coefficients.copy()
features_with_coefficients_abs["coefficients"] = features_with_coefficients_abs.apply(lambda row: abs(row.coefficients), axis=1)
features_with_coefficients_abs
sorted_features = features_with_coefficients_abs.sort_values("coefficients", ascending=False)
top_features = sorted_features.head(20)["feature"].values
x_train[top_features]

print(top_features)
best_indices = [x_train.columns.get_loc(c) for c in top_features]


print(x_train[top_features].columns)

print(x_train.iloc[:, best_indices].columns)
linreg_ranked_features = pr.selectFeatures(x, y, 20, "linreg")
print(x_train.iloc[:, linreg_ranked_features].columns)
linreg_ranked_features[:20]

In [ ]:
# Experiments with separate feature selection per modality
all_features, _ = load.getFeatures()
gen, ge = all_features

# x_train[gen[:2]]
# files
x_train_with_genus_features = x_train[gen]
x_train_with_ge_features = x_train[ge]
# x_train_with_genus_features
x_train_with_ge_features